In [16]:
import os
import sqlite3
import pandas as pd
from tqdm import tqdm
from langchain_community.document_loaders import CSVLoader #
from langchain_core.documents import Document #
from langchain_text_splitters import RecursiveCharacterTextSplitter #
from langchain_ollama import OllamaEmbeddings #
from langchain_chroma import Chroma #


In [17]:
address = pd.read_csv('malaysia-postcodes-location-mukim-district-state.csv')[['address']]
address.to_csv('address.csv', index=False)

C:\Users\izard\AppData\Local\Temp\ipykernel_18524\2952140987.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  address = pd.read_csv('malaysia-postcodes-location-mukim-district-state.csv')[['address']]


- embedding

In [18]:
file_path_csv = 'malaysia-postcodes-location-mukim-district-state.csv'

# Create a CSVLoader instance
loader = CSVLoader(file_path=file_path_csv)
loader

In [19]:
# Load documents from CSV file
documents = loader.load()
documents

[Document(metadata={'source': 'malaysia-postcodes-location-mukim-district-state.csv', 'row': 0}, page_content='postcode: 01000\nlocation: Alor Tok Terang\ndistrict: KANGAR\nstate: PERLIS\nmukim: \naddress: alor tok terang, kangar, 01000, perlis'),
 Document(metadata={'source': 'malaysia-postcodes-location-mukim-district-state.csv', 'row': 1}, page_content='postcode: 01000\nlocation: Bohor Gelam\ndistrict: KANGAR\nstate: PERLIS\nmukim: \naddress: bohor gelam, kangar, 01000, perlis'),
 Document(metadata={'source': 'malaysia-postcodes-location-mukim-district-state.csv', 'row': 2}, page_content='postcode: 01000\nlocation: Jalan Abi Tok Hashim\ndistrict: KANGAR\nstate: PERLIS\nmukim: \naddress: jalan abi tok hashim, kangar, 01000, perlis'),
 Document(metadata={'source': 'malaysia-postcodes-location-mukim-district-state.csv', 'row': 3}, page_content='postcode: 01000\nlocation: Jalan Alor Sena\ndistrict: KANGAR\nstate: PERLIS\nmukim: \naddress: jalan alor sena, kangar, 01000, perlis'),
 Docum

In [20]:
documents[0]

Document(metadata={'source': 'malaysia-postcodes-location-mukim-district-state.csv', 'row': 0}, page_content='postcode: 01000\nlocation: Alor Tok Terang\ndistrict: KANGAR\nstate: PERLIS\nmukim: \naddress: alor tok terang, kangar, 01000, perlis')

In [21]:
documents[0].page_content[:1000]  # Display the first 1000 characters of the first document


'postcode: 01000\nlocation: Alor Tok Terang\ndistrict: KANGAR\nstate: PERLIS\nmukim: \naddress: alor tok terang, kangar, 01000, perlis'

In [22]:
print(len(documents))
total_docs  = len(documents)

165282


In [23]:
# Initialize the embedding
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="llama3.2:1b")

In [52]:
# process each rows 

processed_documents = []

for doc in tqdm(documents):
    source = "https://malaysiapostcode.com/download"
    content_parts = doc.page_content.split('\n')
    address=content_parts[5].replace("address: ", "")

    # Create Document with metadata
    processed_doc = Document(
        page_content=address,
        metadata={"source": source,"postcode": content_parts[0].lower().replace("postcode: ",""), "district": content_parts[2].lower().replace("district: ",""), "state": content_parts[3].lower().replace("state: ",""), "mukim": content_parts[4].lower().replace("mukim: ","")}
    )
    processed_documents.append(processed_doc)


100%|██████████| 165282/165282 [00:01<00:00, 107029.77it/s]


In [55]:
processed_documents[165281]

Document(metadata={'source': 'https://malaysiapostcode.com/download', 'postcode': '94700', 'district': 'samarahan', 'state': 'sarawak', 'mukim': 'bandar terbat'}, page_content='bandar terbat, bandar terbat, samarahan, 94700, sarawak')

In [50]:
len(processed_documents)  # Display the number of processed documents

165282

In [51]:
# Define the folder path for Chroma's in-memory storage
persist_directory = "vectorstore"

In [ ]:
'''
for i in tqdm(range(0, len(processed_documents))):
    vectorstore = Chroma.from_documents(
        documents=[processed_documents[i]], 
        embedding=oembed, 
        persist_directory=persist_directory,
        collection_name="address"  # Specify the collection name here
    )
print('Data Ingested into Vectorstore')'
'''

100%|██████████| 161427/161427 [5:29:31<00:00,  8.16it/s]   

Data Ingested into Vectorstore


In [56]:
for i in tqdm(range(0, len(processed_documents))):
    vectorstore = Chroma.from_documents(
        documents=[processed_documents[i]], 
        embedding=oembed, 
        persist_directory=persist_directory,
        collection_name="address_metadata"  # Specify the collection name here
    )
print('Data Ingested into Vectorstore')

100%|██████████| 165282/165282 [5:37:28<00:00,  8.16it/s]   

Data Ingested into Vectorstore
